In [1]:
import tensorflow as tf
import numpy as np
from sdm_ml.dataset import BBSDataset
from sklearn.preprocessing import StandardScaler
from block_diag import block_diagonal
from kernels import ard_rbf_kernel
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
csv_folder = '../../multi_species/bbs/dataset/csv_bird_data/'
dataset = BBSDataset(csv_folder)

In [3]:
training_set = dataset.get_training_set()

In [4]:
training_set.keys()

dict_keys(['covariates', 'outcomes'])

In [5]:
covariates = training_set['covariates']
outcomes = training_set['outcomes']

scaler = StandardScaler()

# Keep only the first 200 rows and 2 outcomes for this test
covariates = scaler.fit_transform(covariates.values[:200].astype(np.float32))
outcomes = outcomes.values[:200, :2]

In [6]:
covariates.shape

(200, 8)

In [7]:
n_features = covariates.shape[1]
n_outcomes = outcomes.shape[1]
kern_size = covariates.shape[0] * n_outcomes

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_features))
y = tf.placeholder(tf.int64, shape=(None, n_outcomes))

y_flat = tf.cast(tf.reshape(y, (-1,)), tf.float32)

In [8]:
# Next steps:
# 1. Implement the ARD RBF kernel in tensorflow.
# 2. Implement the site kernel in tensorflow (should have derivation). --> Actually, defer this for now and just use the ARD.
# 3. Check that they look reasonable, maybe even on a super-small set.
# 4. Try to optimise with fixed hyperparameters.
# 5. Try to optimise the hyperparameters in an outer loop (Laplace approx.).
# 6. Compare speeds of sparse vs. non-sparse versions.

In [9]:
# Invent some length scales and variances
lengthscales = np.array([
    [4., 5., 2., 8., 3., 9., 5., 3.],
    [2., 8., 9., 3., 3., 4., 9., 7.]
])

alphas = np.array([1., 2.])

tf_scales = tf.Variable(initial_value=lengthscales, dtype=tf.float32)
tf_alphas = tf.Variable(initial_value=alphas, dtype=tf.float32)

In [10]:
ks = list()

# Maybe there would be a more efficient way for large numbers of species than doing it via this list.

for cur_k in range(n_outcomes):
    
    cur_scales = tf_scales[cur_k, :]
    cur_alpha = tf_alphas[cur_k]    
    cur_kernel = ard_rbf_kernel(X, X, cur_scales, cur_alpha)
    ks.append(cur_kernel)
    
block_kern = block_diagonal(ks)

In [11]:
sess = tf.Session()

#block_kernel = sess.run(block_kern, feed_dict={X: covariates})

In [12]:
#plt.matshow(block_kernel)
#plt.colorbar()

In [13]:
f = tf.get_variable('f', shape=(kern_size, 1), initializer=tf.random_normal_initializer(stddev=0.01))

In [14]:
inv_kernel = tf.linalg.inv(block_kern)

In [15]:
log_prior = -0.5 * tf.matmul(tf.matmul(tf.transpose(f), inv_kernel), f)

# Log likelihood
log_lik = tf.reduce_sum(y_flat * f - tf.nn.softplus(f))
log_post = tf.squeeze(log_prior) + log_lik

In [16]:
sess.run(tf.global_variables_initializer())
optimiser = tf.contrib.opt.ScipyOptimizerInterface(
    -log_post, var_list=[f], options={'maxiter': 1000, 'disp': True})

In [17]:
f_best = sess.run(f)

In [18]:
probs = tf.nn.sigmoid(f)
W = -probs * (probs - 1)
W_sqrt = tf.diag(tf.squeeze(tf.sqrt(W)))
multiplied = tf.matmul(tf.matmul(W_sqrt, block_kern), W_sqrt)

# Add on the identity
with_id = tf.eye(kern_size) + multiplied
det_term = tf.linalg.logdet(with_id)

In [19]:
log_marg_lik = tf.squeeze(log_prior) + log_lik - 0.5 * det_term

In [20]:
scale_grad = tf.gradients(log_marg_lik, tf_scales)
alpha_grad = tf.gradients(log_marg_lik, tf_alphas)

step_size = 1e-4

# Take a gradient step in that direction
scale_update = tf.assign(tf_scales, tf_scales + step_size * scale_grad[0])
alpha_update = tf.assign(tf_alphas, tf_alphas + step_size * alpha_grad[0])

for i in range(100):

    optimiser.minimize(sess, feed_dict={X: covariates, y: outcomes})
    print(sess.run(log_marg_lik, feed_dict={X: covariates, y: outcomes}))

    sess.run([scale_update, alpha_update], feed_dict={X: covariates, y: outcomes})
    optimiser.minimize(sess, feed_dict={X: covariates, y: outcomes})

    print(sess.run(log_marg_lik, feed_dict={X: covariates, y: outcomes}))
    
    print(sess.run(tf_scales))
    print(sess.run(tf_alphas))

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15731.326172
  Number of iterations: 49
  Number of functions evaluations: 92
-15735.851
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15732.488281
  Number of iterations: 4
  Number of functions evaluations: 42
-15737.124
[[3.9994464 5.0006375 1.9992473 8.000738  3.00063   8.99932   4.9998975
  3.0006058]
 [2.0013719 7.9973707 8.999717  3.0012605 3.001744  3.9986331 8.999874
  7.001111 ]]
[1.0107065 2.0467331]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15732.488281
  Number of iterations: 1
  Number of functions evaluations: 19
-15737.124
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value:

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15723.791016
  Number of iterations: 2
  Number of functions evaluations: 34
-15728.894
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15724.061523
  Number of iterations: 4
  Number of functions evaluations: 30
-15729.159
[[4.004077  4.9986744 2.003959  8.00285   3.0002859 8.996502  5.000943
  2.9967096]
 [2.0209298 8.004235  8.997076  3.0090113 3.005002  3.9955008 9.003431
  6.9963813]]
[1.0922043 2.2087514]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15724.061523
  Number of iterations: 1
  Number of functions evaluations: 18
-15729.159
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15719.175781
  Number of iterations: 1
  Number of functions evaluations: 16
-15724.432
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15716.808594
  Number of iterations: 5
  Number of functions evaluations: 22
-15722.141
[[4.006255  5.0108695 2.012594  8.007205  2.99869   8.997013  5.0040226
  2.9980288]
 [2.0538578 8.005454  8.994999  3.0146112 3.0217474 3.9967306 9.004293
  6.9973707]]
[1.187008 2.244594]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15716.808594
  Number of iterations: 1
  Number of functions evaluations: 15
-15722.141
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15718.118164
  Number of iterations: 1
  Number of functions evaluations: 18
-15723.558
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15718.382812
  Number of iterations: 4
  Number of functions evaluations: 38
-15723.9375
[[4.0090265 5.017579  2.0099926 8.004333  2.9990218 8.997093  5.004252
  2.9919508]
 [2.046506  8.008687  8.992716  3.0158727 3.017473  4.0077868 9.008029
  6.9898834]]
[1.2047433 2.3407917]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15717.591797
  Number of iterations: 2
  Number of functions evaluations: 11
-15723.146
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15717.510742
  Number of iterations: 1
  Number of functions evaluations: 18
-15723.124
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15716.031250
  Number of iterations: 3
  Number of functions evaluations: 18
-15721.712
[[4.005979  5.019551  2.0032895 8.008021  3.0013347 8.995489  5.0015526
  2.989018 ]
 [2.0620365 8.001892  8.995982  3.0140455 3.0210526 4.0108585 9.001695
  6.9925685]]
[1.2335558 2.3804796]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15716.031250
  Number of iterations: 1
  Number of functions evaluations: 17
-15721.712
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15714.237305
  Number of iterations: 1
  Number of functions evaluations: 17
-15719.825
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15715.540039
  Number of iterations: 2
  Number of functions evaluations: 20
-15721.096
[[4.0086303 5.021457  2.012084  8.011431  2.9995513 8.99454   5.007906
  2.9844527]
 [2.0640323 7.999647  8.995565  3.0023987 3.0225327 4.0093374 9.00061
  6.992566 ]]
[1.24287  2.306078]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15715.540039
  Number of iterations: 1
  Number of functions evaluations: 15
-15721.096
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1571

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15714.750000
  Number of iterations: 2
  Number of functions evaluations: 27
-15720.051
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15714.118164
  Number of iterations: 5
  Number of functions evaluations: 17
-15719.382
[[4.0077453 5.0186443 2.014995  8.01022   2.9957073 8.995163  5.004434
  2.9830198]
 [2.0502732 8.00334   8.998951  2.987248  3.0064547 4.0059586 9.003736
  6.985228 ]]
[1.3011736 2.0933838]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15714.118164
  Number of iterations: 1
  Number of functions evaluations: 14
-15719.382
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15712.529297
  Number of iterations: 1
  Number of functions evaluations: 12
-15718.021
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15712.994141
  Number of iterations: 5
  Number of functions evaluations: 75
-15718.552
[[4.0092363 5.0168757 2.0234747 8.013533  3.0031598 8.994726  5.0026727
  2.9869528]
 [2.045067  8.013123  8.996887  2.9973037 3.014993  4.0131106 9.005904
  6.9803195]]
[1.397863  2.1499958]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15712.994141
  Number of iterations: 1
  Number of functions evaluations: 18
-15718.552
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 

In [22]:
lengthscales

array([[4., 5., 2., 8., 3., 9., 5., 3.],
       [2., 8., 9., 3., 3., 4., 9., 7.]])